# Quickstart

Get started with AgentOps in minutes with just 2 lines of code for basic monitoring, and explore powerful decorators for custom tracing.

AgentOps is designed for easy integration into your AI agent projects, providing powerful observability with minimal setup. This guide will get you started quickly.

## Installation

First, install the AgentOps SDK. We recommend including `python-dotenv` for easy API key management. `uv add agentops python-dotenv`

## Initial Setup (2 Lines of Code)

At its simplest, AgentOps can start monitoring your supported LLM and agent framework calls with just two lines of Python code.

1. **Import AgentOps**: Add `import agentops` to your script.
2. **Initialize AgentOps**: Call `agentops.init()` with your API key.

## Setting Your AgentOps API Key
You need an AgentOps API key to send data to your dashboard.

- Get your API key from the [AgentOps Dashboard](https://app.agentops.ai/settings/projects).

It’s best practice to set your API key as an environment variable.

```bash
export AGENTOPS_API_KEY="your_agentops_api_key_here"
```

If you use a `.env` file, make sure `load_dotenv()` is called before `agentops.init()`.

In [1]:
import os
import agentops
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Initialize AgentOps
agentops.init(os.getenv("AGENTOPS_API_KEY"))

# That's it for basic auto-instrumentation!
# If you're using a supported library (like OpenAI, LangChain, CrewAI, etc.),
# AgentOps will now automatically track LLM calls and agent actions.

🖇 AgentOps: You're on the agentops free plan 🤔


## Running Your Agent & Viewing Traces
After adding the two lines and ensuring your API key is set up:

1. Run your agent application as you normally would.
2. AgentOps will automatically instrument supported libraries and send trace data.
3. Visit your [AgentOps Dashboard](https://app.agentops.ai/traces) to observe your agent’s operations!

## Beyond Automatic Instrumentation: Decorators

While AgentOps automatically instruments many popular libraries, you can gain finer-grained control and track custom parts of your code using our powerful decorators. This allows you to define specific operations, group logic under named agents, track tool usage with costs, and create custom traces.

### Tracking Custom Operations with @operation

Instrument any function in your code to create spans that track its execution, parameters, and return values. These operations will appear in your session visualization alongside LLM calls.

![img](imgs/1.task.png)

In [2]:
from agentops.sdk.decorators import operation

@operation
def process_data(data:str):
    # Your function logic here
    processed_result = data.upper()
    # agentops.record(Events("Processed Data", result=processed_result)) # Optional: record specific events
    return processed_result

In [3]:
# Example usage:
my_data = "example input"
output = process_data(my_data)

```json
{
  "freeplan_truncated": false,
  "span_id": "4115fe73a6a6359f",
  "parent_span_id": "",
  "span_name": "process_data.task",
  "span_kind": "Internal",
  "span_type": "other",
  "service_name": "agentops",
  "start_time": "2025-08-04T02:46:02.902000+00:00",
  "end_time": "2025-08-04T02:46:02.902046+00:00",
  "duration": 46779,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "5d10e608-ac69-4298-98f4-21922b52982a",
    "agentops": {
      "project": {
        "id": "5d10e608-ac69-4298-98f4-21922b52982a"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "span": {
        "kind": "task"
      },
      "task": {
        "input": "{\"args\": [\"example input\"], \"kwargs\": {}}",
        "output": "EXAMPLE INPUT"
      }
    },
    "operation": {
      "name": "process_data"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": null
}
```

### Tracking Agent Logic with @agent

If you structure your system with specific named agents (e.g., classes), use the `@agent` decorator on the class and `@operation` on its methods to group all downstream operations under that agent’s context.

In [4]:
from agentops.sdk.decorators import agent, operation


@agent(name="MyCustomAgent") # You can provide a name for the agent
class MyAgent:
    def __init__(self, agent_id:str):
        self.agent_id = agent_id # agent_id is a reserved parameter for AgentOps

    @operation
    def perform_task(self, task_description:str):
        # Agent task logic here
        # This could include LLM calls or calls to other @operation decorated functions
        return f"Agent {self.agent_id} completed: {task_description}"

# Example usage
research_agent = MyAgent(agent_id="researcher-001")
result = research_agent.perform_task("Analyze market trends")

```json
{
  "freeplan_truncated": false,
  "span_id": "0b527e1e67235d6f",
  "parent_span_id": "7a233418d6748327",
  "span_name": "perform_task.task",
  "span_kind": "Internal",
  "span_type": "other",
  "service_name": "agentops",
  "start_time": "2025-08-04T02:57:04.854771+00:00",
  "end_time": "2025-08-04T02:57:04.854791+00:00",
  "duration": 20818,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "5d10e608-ac69-4298-98f4-21922b52982a",
    "agentops": {
      "project": {
        "id": "5d10e608-ac69-4298-98f4-21922b52982a"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "span": {
        "kind": "task"
      },
      "task": {
        "input": "{\"args\": [\"Analyze market trends\"], \"kwargs\": {}}",
        "output": "Agent researcher-001 completed: Analyze market trends"
      }
    },
    "operation": {
      "name": "perform_task"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": null
}
```

### Tracking Tools with @tool

Track the usage of specific tools or functions, and optionally associate costs with them. This data will be aggregated in your dashboard.

![img](imgs/2.tool.png)

In [5]:
from agentops.sdk.decorators import tool

@tool(name="WebSearchTool", cost=0.05) # Cost is optional
def web_search(query:str)->str:
    # Tool logic here
    return f"Search results for {query}"

@tool # No cost specified
def calculator(expression:str)->str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error {e}"

In [6]:
# Example usage:
search_result = web_search("AgentOps features")

In [7]:
calculation = calculator("2 + 2")

### Grouping with Traces (@trace or manual)

Create custom traces to group a sequence of operations or define logical units of work. You can use the `@trace` decorator or manage traces manually for more complex scenarios.

**Note**: *If `auto_start_session=False` in `agentops.init()`, you must use `@trace` or `agentops.start_trace()` for any data to be recorded.*

![img](imgs/3.session.png)

In [8]:
from agentops.sdk.decorators import trace
# Assuming MyAgent and web_search are defined as above

# Option 1: Using the @trace decorator
@trace(name="MyMainWorkflow", tags=["main-flow"])
def my_workflow_decorated(task_to_perform):
    # Your workflow code here
    # @trace -> @agent -> @perform
    main_agent = MyAgent(agent_id="workflow-agent")
    result = main_agent.perform_task(task_to_perform)
    # Example of using a tool within the trace
    tool_result = web_search(f"details for {task_to_perform}")
    return result, tool_result

In [9]:
result_decorated = my_workflow_decorated("complex data procesing")

🖇 AgentOps: Session Replay for MyMainWorkflow trace: https://app.agentops.ai/sessions?trace_id=2d110530b84609d07bbfce62d7e975d7
🖇 AgentOps: Session Replay for MyMainWorkflow.session trace: https://app.agentops.ai/sessions?trace_id=2d110530b84609d07bbfce62d7e975d7


In [10]:
# Option 2: Managing traces manually
# import agentops # Already imported

# custom_trace = agentops.start_trace(name="MyManualWorkflow", tags=["manual-flow"])
# try:
#     # Your code here
#     main_agent = MyAgent(agent_id="manual-workflow-agent") # Assuming MyAgent is defined
#     result = main_agent.perform_task("another complex task")
#     tool_result = web_search(f"info for {result}") # Assuming web_search is defined
#     agentops.end_trace(custom_trace, end_state="Success", end_prompt=f"Completed: {result}")
# except Exception as e:
#     if custom_trace: # Ensure trace was started before trying to end it
#         agentops.end_trace(custom_trace, end_state="Fail", error_message=str(e))
#     raise

Session:

```json
{
  "freeplan_truncated": false,
  "span_id": "506866e81c9496d7",
  "parent_span_id": "5f13c67e29b720c6",
  "span_name": "MyMainWorkflow.session",
  "span_kind": "Internal",
  "span_type": "other",
  "service_name": "agentops",
  "start_time": "2025-08-04T04:18:36.555685+00:00",
  "end_time": "2025-08-04T04:18:36.556612+00:00",
  "duration": 927871,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "540dbfef-806a-4ef8-973a-87d7d94eb7be",
    "agentops": {
      "project": {
        "id": "540dbfef-806a-4ef8-973a-87d7d94eb7be"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "entity": {
        "input": "{\"args\": [\"complex data procesing\"], \"kwargs\": {}}",
        "output": "[\"Agent workflow-agent completed: complex data procesing\", \"Search results for details for complex data procesing\"]"
      },
      "session": {
        "end_state": "Success"
      },
      "span": {
        "kind": "session"
      },
      "tags": "[\"main-flow\"]"
    },
    "operation": {
      "name": "MyMainWorkflow"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": null
}
```

Agent:
```json
{
  "freeplan_truncated": false,
  "span_id": "272149f96805bd50",
  "parent_span_id": "506866e81c9496d7",
  "span_name": "MyCustomAgent.agent",
  "span_kind": "Internal",
  "span_type": "other",
  "service_name": "agentops",
  "start_time": "2025-08-04T04:18:36.556438+00:00",
  "end_time": "2025-08-04T04:18:36.556586+00:00",
  "duration": 148210,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "540dbfef-806a-4ef8-973a-87d7d94eb7be",
    "agentops": {
      "project": {
        "id": "540dbfef-806a-4ef8-973a-87d7d94eb7be"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "entity": {
        "input": "{\"args\": [], \"kwargs\": {\"agent_id\": \"workflow-agent\"}}"
      },
      "span": {
        "kind": "agent"
      }
    },
    "operation": {
      "name": "MyCustomAgent"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": null
}
```

Perform

```json
{
  "freeplan_truncated": false,
  "span_id": "6e4e5025bf5e8f22",
  "parent_span_id": "272149f96805bd50",
  "span_name": "perform_task.task",
  "span_kind": "Internal",
  "span_type": "other",
  "service_name": "agentops",
  "start_time": "2025-08-04T04:18:36.556499+00:00",
  "end_time": "2025-08-04T04:18:36.556515+00:00",
  "duration": 16300,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "540dbfef-806a-4ef8-973a-87d7d94eb7be",
    "agentops": {
      "project": {
        "id": "540dbfef-806a-4ef8-973a-87d7d94eb7be"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "span": {
        "kind": "task"
      },
      "task": {
        "input": "{\"args\": [\"complex data procesing\"], \"kwargs\": {}}",
        "output": "Agent workflow-agent completed: complex data procesing"
      }
    },
    "operation": {
      "name": "perform_task"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": null
}
```

Tool:
```json
{
  "freeplan_truncated": false,
  "span_id": "33f4558abb95c924",
  "parent_span_id": "272149f96805bd50",
  "span_name": "WebSearchTool.tool",
  "span_kind": "Internal",
  "span_type": "tool",
  "service_name": "agentops",
  "start_time": "2025-08-04T04:18:36.556560+00:00",
  "end_time": "2025-08-04T04:18:36.556573+00:00",
  "duration": 13984,
  "status_code": "UNSET",
  "status_message": "",
  "attributes": {},
  "resource_attributes": {
    "ProjectId": "540dbfef-806a-4ef8-973a-87d7d94eb7be",
    "agentops": {
      "project": {
        "id": "540dbfef-806a-4ef8-973a-87d7d94eb7be"
      }
    },
    "host": {
      "name": "e68e96fc8302"
    },
    "imported_libraries": "[\"agentops\"]",
    "os": {
      "type": "linux"
    },
    "service": {
      "name": "agentops"
    }
  },
  "span_attributes": {
    "agentops": {
      "span": {
        "kind": "tool"
      },
      "tool": {
        "input": "{\"args\": [\"details for complex data procesing\"], \"kwargs\": {}}",
        "output": "Search results for details for complex data procesing"
      }
    },
    "gen_ai": {
      "usage": {
        "total_cost": "0.05"
      }
    },
    "operation": {
      "name": "WebSearchTool"
    }
  },
  "event_timestamps": [],
  "event_names": [],
  "event_attributes": [],
  "link_trace_ids": [],
  "link_span_ids": [],
  "link_trace_states": [],
  "link_attributes": [],
  "metrics": {
    "total_tokens": 0,
    "prompt_tokens": 0,
    "completion_tokens": 0,
    "cache_read_input_tokens": 0,
    "reasoning_tokens": 0,
    "success_tokens": 0,
    "fail_tokens": 0,
    "indeterminate_tokens": 0,
    "prompt_cost": "0.0000000",
    "completion_cost": "0.0000000",
    "total_cost": "0.0500000"
  }
}
```

### Updating Trace Metadata

You can also update metadata on running traces to add context or track progress:

In [14]:
from agentops import update_trace_metadata

# Update metadata during trace execution
update_trace_metadata({
    "operation_name": "AI Agent Processing",
    "processing_stage": "data_validation",
    "records_processed": 1500,
    "user_id": "user_123",
    "tags": ["validation", "production"]
})

True

## Complete Example with Decorators

Here’s a consolidated example showcasing how these decorators can work together:

![img](imgs/4.workflow.png)

In [1]:
import agentops
from agentops.sdk.decorators import agent, operation, tool, trace
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY")

# Initialize AgentOps. 
# Set auto_start_session=False because @trace will manage the session.
agentops.init(AGENTOPS_API_KEY, auto_start_session=False, tags=["quickstart-complete-example"])

# Define a tool
@tool(name="AdvancedSearch", cost=0.02)
def advanced_web_search(query: str) -> str:
    # Simulate a more advanced search
    return f"Advanced search results for '{query}': [Details...]"

# Define an agent class
@agent(name="ResearchSpecialistAgent")
class ResearchAgent:
    def __init__(self, agent_id: str):
        self.agent_id = agent_id # This will be used as the agent_id in AgentOps
        
    @operation(name="ConductResearch")
    def conduct_research(self, research_topic: str) -> str:
        # Use the tool within the agent's operation
        search_results = advanced_web_search(f"Deep dive into {research_topic}")
        # Simulate further processing
        analysis = f"Analysis of '{research_topic}': Based on '{search_results}', the key findings are..."
        return analysis

# Define a workflow using the @trace decorator
@trace(name="FullResearchWorkflow", tags=["research", "analysis", "example"])
def run_full_research_workflow(topic: str) -> str:
    specialist_agent = ResearchAgent(agent_id="researcher-alpha-007")
    research_findings = specialist_agent.conduct_research(topic)
    
    final_report = f"Research Report for '{topic}':\n{research_findings}"
    # agentops.record(Events("ReportGenerated", details=final_report)) # Optional: record a custom event
    return final_report

# Execute the workflow
final_output = run_full_research_workflow("AI in healthcare")
print(final_output)

🖇 AgentOps: You're on the agentops free plan 🤔
🖇 AgentOps: Session Replay for FullResearchWorkflow trace: https://app.agentops.ai/sessions?trace_id=b8ca7c097cbdee0ad785e589dd3a5301
🖇 AgentOps: Session Replay for FullResearchWorkflow.session trace: https://app.agentops.ai/sessions?trace_id=b8ca7c097cbdee0ad785e589dd3a5301


Research Report for 'AI in healthcare':
Analysis of 'AI in healthcare': Based on 'Advanced search results for 'Deep dive into AI in healthcare': [Details...]', the key findings are...
